# Edge Condition - Conditional Routing with Structured Outputs

## Overview
This notebook demonstrates **conditional routing** in workflows using edge conditions. You'll learn how to:
- Attach boolean predicates to edges for if/else routing logic
- Use Pydantic models for structured agent outputs (JSON validation)
- Implement spam classification with conditional handling
- Transform agent responses between workflow stages
- Route based on data inspection rather than hardcoded paths

## What This Sample Does
The workflow implements an **email triage system** with conditional routing:
1. **spam_detection_agent** - Analyzes email, returns `DetectionResult` (JSON)
2. **Conditional Routing:**
   - **If NOT spam** → Transform to `AgentExecutorRequest` → `email_assistant_agent` → Draft reply
   - **If spam** → `handle_spam_classifier_response` → Block email

**Input:** Email text (from `resources/email.txt`)  
**Output:** Either drafted reply OR spam notice (based on classification)

## Key Concepts
- **Edge Conditions:** Predicate functions that inspect messages to decide routing
- **Pydantic Models:** Type-safe structured outputs (`DetectionResult`, `EmailResponse`)
- **Response Format:** `response_format` parameter forces LLMs to return valid JSON
- **Transformer Executors:** Convert between data types (response → request)
- **Defensive Parsing:** Handle parse errors gracefully to avoid dead ends

## Prerequisites
**Required:**
- Azure OpenAI access configured for `AzureOpenAIChatClient`
- Azure CLI authentication: Run `az login` before executing
- Sample email file at `workflow/resources/email.txt` (or use fallback)

**Concepts:**
- Familiarity with `WorkflowBuilder`, executors, edges
- Understanding of edge conditions and predicates
- Basic knowledge of Pydantic models

## Step 1: Import Required Libraries

In [ ]:
# Copyright (c) Microsoft. All rights reserved.

from dotenv import load_dotenv
import asyncio
import os
from typing import Any

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    executor,
)
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
from pydantic import BaseModel
from typing_extensions import Never

load_dotenv('../../.env')

endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")


## Step 2: Define Pydantic Models for Structured Outputs
These models ensure agents return valid, parseable JSON.

**DetectionResult:**
- `is_spam` - Boolean driving routing decisions
- `reason` - Human-readable rationale
- `email_content` - Original email (passed downstream)

**EmailResponse:**
- `response` - Drafted reply text

In [ ]:
class DetectionResult(BaseModel):
    """Represents the result of spam detection."""

    # is_spam drives the routing decision taken by edge conditions
    is_spam: bool
    # Human readable rationale from the detector
    reason: str
    # The agent must include the original email so downstream agents can operate without reloading content
    email_content: str


class EmailResponse(BaseModel):
    """Represents the response from the email assistant."""

    # The drafted reply that a user could copy or send
    response: str

## Step 3: Create Condition Factory
This function generates edge predicates that route based on `is_spam`.

**How It Works:**
1. `get_condition(expected_result)` returns a predicate function
2. Predicate receives upstream message (any type)
3. Validates message is `AgentExecutorResponse`
4. Parses JSON into `DetectionResult` model
5. Returns `True` if `is_spam` matches `expected_result`

**Defensive Programming:**
- Returns `True` for non-`AgentExecutorResponse` to avoid dead ends
- Returns `False` on parse errors (fail-closed)

In [ ]:
def get_condition(expected_result: bool):
    """Create a condition callable that routes based on DetectionResult.is_spam."""

    # The returned function will be used as an edge predicate.
    # It receives whatever the upstream executor produced.
    def condition(message: Any) -> bool:
        # Defensive guard. If a non AgentExecutorResponse appears, let the edge pass to avoid dead ends.
        if not isinstance(message, AgentExecutorResponse):
            return True

        try:
            # Prefer parsing a structured DetectionResult from the agent JSON text.
            # Using model_validate_json ensures type safety and raises if the shape is wrong.
            detection = DetectionResult.model_validate_json(message.agent_run_response.text)
            # Route only when the spam flag matches the expected path.
            return detection.is_spam == expected_result
        except Exception:
            # Fail closed on parse errors so we do not accidentally route to the wrong path.
            # Returning False prevents this edge from activating.
            return False

    return condition

## Step 4: Define Executor Functions
Create lightweight executors using `@executor` decorator.

**handle_email_response:**
- Terminal executor for NOT spam path
- Parses `EmailResponse` JSON and yields output

**handle_spam_classifier_response:**
- Terminal executor for spam path
- Validates `is_spam=True` and yields spam notice

**to_email_assistant_request:**
- Transformer executor
- Converts `AgentExecutorResponse` → `AgentExecutorRequest`
- Extracts `email_content` from `DetectionResult`
- Forwards as new user message to email assistant

In [ ]:
@executor(id="send_email")
async def handle_email_response(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    # Downstream of the email assistant. Parse a validated EmailResponse and yield the workflow output.
    email_response = EmailResponse.model_validate_json(response.agent_run_response.text)
    await ctx.yield_output(f"Email sent:\n{email_response.response}")


@executor(id="handle_spam")
async def handle_spam_classifier_response(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    # Spam path. Confirm the DetectionResult and yield the workflow output. Guard against accidental non spam input.
    detection = DetectionResult.model_validate_json(response.agent_run_response.text)
    if detection.is_spam:
        await ctx.yield_output(f"Email marked as spam: {detection.reason}")
    else:
        # This indicates the routing predicate and executor contract are out of sync.
        raise RuntimeError("This executor should only handle spam messages.")


@executor(id="to_email_assistant_request")
async def to_email_assistant_request(
    response: AgentExecutorResponse, ctx: WorkflowContext[AgentExecutorRequest]
) -> None:
    """Transform detection result into an AgentExecutorRequest for the email assistant.

    Extracts DetectionResult.email_content and forwards it as a user message.
    """
    # Bridge executor. Converts a structured DetectionResult into a ChatMessage and forwards it as a new request.
    detection = DetectionResult.model_validate_json(response.agent_run_response.text)
    user_msg = ChatMessage(Role.USER, text=detection.email_content)
    await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))

## Step 5: Build the Conditional Workflow
Create agents and wire them with conditional edges.

**Workflow Graph:**
```
                    spam_detection_agent
                            │
                ┌───────────┴───────────┐
                │ (condition check)     │
         is_spam=False           is_spam=True
                │                      │
                ▼                      ▼
    to_email_assistant_request   handle_spam
                │
                ▼
      email_assistant_agent
                │
                ▼
        handle_email_response
```

**Key Points:**
- `response_format=DetectionResult` forces structured JSON
- `condition=get_condition(False)` routes NOT spam emails
- `condition=get_condition(True)` routes spam emails

In [ ]:
async def run_workflow():
    """Main function to build and run the conditional routing workflow."""
    # Create agents
    # AzureCliCredential uses your current az login. This avoids embedding secrets in code.
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
    chat_client = AzureOpenAIChatClient(
        deployment_name=deployment_name,
        endpoint=endpoint,
        credential=AzureCliCredential()
    )

    # Agent 1. Classifies spam and returns a DetectionResult object.
    # response_format enforces that the LLM returns parsable JSON for the Pydantic model.
    spam_detection_agent = AgentExecutor(
        chat_client.create_agent(
            instructions=(
                "You are a spam detection assistant that identifies spam emails. "
                "Always return JSON with fields is_spam (bool), reason (string), and email_content (string). "
                "Include the original email content in email_content."
            ),
            response_format=DetectionResult,
        ),
        id="spam_detection_agent",
    )

    # Agent 2. Drafts a professional reply. Also uses structured JSON output for reliability.
    email_assistant_agent = AgentExecutor(
        chat_client.create_agent(
            instructions=(
                "You are an email assistant that helps users draft professional responses to emails. "
                "Your input may be a JSON object that includes 'email_content'; base your reply on that content. "
                "Return JSON with a single field 'response' containing the drafted reply."
            ),
            response_format=EmailResponse,
        ),
        id="email_assistant_agent",
    )

    # Build the workflow graph.
    # Start at the spam detector.
    # If not spam, hop to a transformer that creates a new AgentExecutorRequest,
    # then call the email assistant, then finalize.
    # If spam, go directly to the spam handler and finalize.
    workflow = (
        WorkflowBuilder()
        .set_start_executor(spam_detection_agent)
        # Not spam path: transform response -> request for assistant -> assistant -> send email
        .add_edge(spam_detection_agent, to_email_assistant_request, condition=get_condition(False))
        .add_edge(to_email_assistant_request, email_assistant_agent)
        .add_edge(email_assistant_agent, handle_email_response)
        # Spam path: send to spam handler
        .add_edge(spam_detection_agent, handle_spam_classifier_response, condition=get_condition(True))
        .build()
    )

    # Read Email content from the sample resource file.
    # This keeps the sample deterministic since the model sees the same email every run.
    email_path = os.path.join(os.path.dirname(os.path.dirname(os.path.realpath(__file__))), "resources", "email.txt")

    try:
        with open(email_path) as email_file:  # noqa: ASYNC230
            email = email_file.read()
    except FileNotFoundError:
        # Fallback email for demo purposes
        email = """Subject: Team Meeting Follow-up - Action Items

Hi Sarah,

I wanted to follow up on our team meeting this morning and share the action items we discussed:

1. Update the project timeline by Friday
2. Schedule client presentation for next week
3. Review the budget allocation for Q4

Please let me know if you have any questions.

Best regards,
Alex Johnson"""

    print("=== Email Content ===")
    print(email[:200] + "..." if len(email) > 200 else email)
    print("\n=== Processing ===")

    # Execute the workflow. Since the start is an AgentExecutor, pass an AgentExecutorRequest.
    # The workflow completes when it becomes idle (no more work to do).
    request = AgentExecutorRequest(messages=[ChatMessage(Role.USER, text=email)], should_respond=True)
    events = await workflow.run(request)
    outputs = events.get_outputs()
    if outputs:
        print(f"\n=== Workflow Output ===")
        print(outputs[0])

## Step 6: Execute the Workflow
**Note:** Requires Azure OpenAI credentials. Authenticate with `az login`.

In [ ]:
# Uncomment to run (requires Azure OpenAI access)
# await run_workflow()

## Expected Output (NOT Spam Path)
```
=== Email Content ===
Subject: Team Meeting Follow-up - Action Items

Hi Sarah,

I wanted to follow up on our team meeting this morning...

=== Processing ===

=== Workflow Output ===
Email sent:
Hi Alex,

Thank you for the follow-up and for summarizing the action items from this morning's meeting. 
The points you listed accurately reflect our discussion, and I don't have any additional items to add.
I will update the project timeline by Friday, schedule the client presentation for next week, 
and review the Q4 budget allocation.

Best regards,
Sarah
```

## Key Takeaways

### Conditional Routing Pattern
✅ **Edge Conditions:** Predicates inspect messages to decide routing  
✅ **If/Else Logic:** `condition=get_condition(False)` vs `condition=get_condition(True)`  
✅ **Data-Driven Routing:** Decisions based on message content, not hardcoded paths  
✅ **Multiple Paths:** Same upstream executor can route to different downstreams

### Structured Outputs with Pydantic
**Benefits:**
- **Type Safety:** Catch schema mismatches at parse time
- **Validation:** Automatic field validation (types, required fields)
- **IDE Support:** Autocomplete and type hints
- **Documentation:** Models serve as schema documentation

**How It Works:**
1. Define Pydantic models (`DetectionResult`, `EmailResponse`)
2. Pass to agent: `response_format=DetectionResult`
3. LLM returns JSON matching the schema
4. Parse safely: `model_validate_json(response.text)`

### Transformer Executors
**Purpose:** Convert between data types to bridge executors

**Example:** `to_email_assistant_request`
```python
# Input: AgentExecutorResponse (DetectionResult JSON)
# Output: AgentExecutorRequest (new user message)
detection = DetectionResult.model_validate_json(response.text)
user_msg = ChatMessage(Role.USER, text=detection.email_content)
await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
```

**Common Transformations:**
- Response → Request (chain agents)
- Structured → Primitive (extract fields)
- Primitive → Structured (wrap in models)

### Defensive Programming
**Avoid Dead Ends:**
```python
if not isinstance(message, AgentExecutorResponse):
    return True  # Let edge pass to avoid workflow stalling
```

**Fail-Closed on Errors:**
```python
try:
    detection = DetectionResult.model_validate_json(response.text)
    return detection.is_spam == expected_result
except Exception:
    return False  # Don't route on parse errors
```

## Conditional Routing Patterns

### Binary Decision (If/Else)
```python
workflow.add_edge(classifier, path_a, condition=lambda msg: msg.category == 'A')
workflow.add_edge(classifier, path_b, condition=lambda msg: msg.category == 'B')
```

### Multi-Way Branching (If/Elif/Else)
```python
# Edges evaluated in order until one matches
workflow.add_edge(classifier, urgent, condition=lambda m: m.priority == 'urgent')
workflow.add_edge(classifier, normal, condition=lambda m: m.priority == 'normal')
workflow.add_edge(classifier, low, condition=lambda m: True)  # Default/fallback
```

### Guard Conditions
```python
# Only proceed if validation passes
workflow.add_edge(validator, processor, condition=lambda m: m.is_valid)
workflow.add_edge(validator, error_handler, condition=lambda m: not m.is_valid)
```

## Workflow Execution Flow

### NOT Spam Path:
1. `spam_detection_agent` receives email
2. Returns `DetectionResult(is_spam=False, reason="...", email_content="...")`
3. `get_condition(False)` evaluates to `True` → routes to transformer
4. `to_email_assistant_request` extracts `email_content`, creates new request
5. `email_assistant_agent` drafts reply
6. `handle_email_response` parses `EmailResponse`, yields output

### Spam Path:
1. `spam_detection_agent` receives email
2. Returns `DetectionResult(is_spam=True, reason="Suspicious content", ...)`
3. `get_condition(True)` evaluates to `True` → routes to spam handler
4. `handle_spam_classifier_response` yields spam notice

## Best Practices
- **Use Pydantic models** for agent outputs to ensure type safety
- **Set `response_format`** to enforce structured JSON from LLMs
- **Handle parse errors** gracefully in condition functions
- **Validate executor inputs** with type checks and assertions
- **Provide fallback paths** to prevent workflow deadlocks
- **Keep conditions simple** - complex logic belongs in executors

## Next Steps
- See `switch_case_edge_group.ipynb` for cleaner multi-way routing
- Explore `multi_selection_edge_group.ipynb` for parallel branching (fan-out)
- Check `simple_loop.ipynb` for iterative patterns with feedback loops